Régression linéaire
===================

L'objectif de ce TP est d'effectuer les différents calculs des notions vues en cours sur le modèle
$$Y = X\beta + \varepsilon$$
===========================
dans le cas où $X$ est ou non de plein rang.

On commence par importer les librairies dont on aura besoin pour le TP, à savoir *numpy* pour les outils mathématiques et *sklearn* pour les outils d'apprentissage (LASSO, Ridge, Elastic net).

Pour que les *plot* s'affichent dans la page, on ajoute

    %matplotlib inline

In [ ]:
%matplotlib inline
from sklearn import linear_model
import numpy as np
import matplotlib.pyplot as plt

 ## Méthode des moindres carrés ordinaire
 
 Dans un premier temps, nous allons générer $Y$ en fonction de $X$ à partir d'un $\beta$ que l'on se fixe.
 

In [ ]:
n=50
beta=[1,4,1,-7,-2,1,0.7,0.1,3,0.01]
X=np.random.randn(n,10)
eps=np.random.normal(0,1,n)
Y=np.dot(X,beta)+eps
print(Y)

L'objectif est de retrouver $\beta$ à partir de l'observation de $X$ et $Y$.

Dans un premier, on affiche sur un graphe $Y$ en fonction d'une coordonnée de $X$.

In [ ]:
fig, ax = plt.subplots()
ax.scatter(Y, X[:,8], alpha=0.5)

ax.set_xlabel(r'$X$', fontsize=20)
ax.set_ylabel(r'$Y$', fontsize=20)
ax.set_title(r'$Y$ en fonction de $X_9\beta_9$')

ax.grid(True)
fig.tight_layout()

plt.show()

On va avoir du mal à prédire $Y$ seulement à partir de $X_9$.
On calcule l'erreur quadratique moyenne dans cette régression de $Y$ sur $X_9$ lorsque $\beta_9=3$ est connu.

Pour cela, on utilise

    np.linalg.norm

In [ ]:
#help(np.linalg.norm)
r1=np.linalg.norm(Y-3*X[:,8])/n
r2=np.linalg.norm(Y-np.dot(X,beta))/n
print([r1,r2])

**Calculer l'erreur quadratique moyenne de la régression de $Y$ sur $X$ lorsque $\beta$ est connu.**

Comme attendu, l'écart quadratique moyen est inférieur lorsque l'on régresse $Y$ sur toute les colonnes de $X$.
Autrement dit, en utilisant toute l'information que l'on a sur $Y$ (i.e. toute les colonnes de $X$), on fait mieux.

Maintenant, on suppose que $\beta$ est inconnue. 
On veut le retrouver à partir de nos observations de $X$ et $Y$.

**Estimer $\beta$ en appliquant la méthode des moindres carrés.**

On pourra utiliser les méthodes

    np.linalg.inv
    np.linalg.dot
    np.transpose

In [ ]:
help(np.linalg.inv)
help(np.linalg.dot)
help(np.transpose)

La variance de $\varepsilon$ est $1$. À partir de la formule du cours, **donner la variance des $\hat\beta_i$ (conditionnellement à $X$).**

On pourra utiliser 

        np.diag

In [ ]:
help(np.diag)

**En déduire un intervalle de confiance à 0.95 des $\beta_j$.
Les *vrais* $\beta_j$ sont-ils dans cet intervalle?**

Pour rappel, le quantile d'ordre 0.975 de la loi normale est $\approx$ 1.96

### Cas parcimonieux

On reprend maintenant le même contexte, mais cette fois, des colonnes sont ajoutées à $X$. Autrement dit, l'information qui nous permet d'estimer $X$ (i.e. les colonnes $j$ de $X$ ayant un grand coefficient $\beta_j$) est masquée parmi un grand nombre de colonnnes de $X$.
On note alors $\beta'$ ce nouveau vecteur.

*Pour les modèles avec parcimonie, on notera $k$ le nombre de coordonnées non nulles de $\beta$.*

In [ ]:
d=7000
k=len(beta)
betap=np.concatenate((beta,np.zeros(d-k)))
np.random.shuffle(betap)
print(betap)
Xp=np.random.randn(n,d)
Yp=np.dot(Xp,betap)+eps

**Que renvoie la méthode des moindres carrés?
Commenter**

*Enregistrer votre travail avant d'executer la cellule suivante*

In [ ]:
hatbetap=np.dot(np.dot(np.linalg.inv(np.dot(np.transpose(Xp),Xp)),np.transpose(Xp)),Y)
print(hatbetap)
hatbetap-betap

Pour pouvoir retrouver nos coefficients, on va utiliser la méthode LASSO dans un premier temps.

## Méthode LASSO

La méthode de régression LASSO est implémentée dans la librairie *scikitlearn* via la classe

    linear_model.Lasso
   
Nous avons vu qu'il est possible d'ajouter une colonne de $1$ à $X$ pour des données non centrées. Cette classe permet d'automatiser l'ajout de cette colonne, qui est alors appelée *intercept*.
Comme on sait que dans notre modèle, il n'y a pas de colonne de 1 et que cette classe l'ajoute par défaut, on va lui demander de ne pas l'ajouter via l'argument *intercept=False*

*Dans cette classe, le $\lambda$ du cours est appelé* alpha

On utilise la valeur $\lambda=8\sqrt{\frac{2d}{n}}$.

In [ ]:
l=8*np.sqrt(np.log(2*d)/n)
clf = linear_model.Lasso(alpha=l,fit_intercept=False)
clf.fit(Xp, Yp)                                        #effecture les calculs pour estimer beta
hatbetap=clf.coef_                                     #renvoie les coefficients estimés par la méthode LASSO

print(betap[betap!=0])
print(betap[hatbetap!=0])
print(hatbetap[betap!=0])
print(np.linalg.norm(Yp-np.dot(Xp,hatbetap))/n,10*np.log(2*d)/n)

**Commenter. Est-ce que la méthode semble bien fonctionner?**

*Réponse:*

## Méthode avec pénalisation $|\hspace{0.3cm}|_0$.

Nous avons vu qu'il est possible de pénaliser la minimisation de $\beta \mapsto \|Y-X\beta\|$ par $+\lambda |\beta|_0$ pour chercher les coordonnées non nulles de $\beta$.
Ou de manière équivalente, on peut minimiser $\beta \mapsto \|Y-X\beta\|$ sous la contrainte $|\beta|_0\leq k$.

**Implémenter cette minimisation pour $k=3$**

On pourra utiliser 

    np.inf
    np.argmin
    np.unravel_index
    
**Puis tester pour $d=20,50,100$**

In [ ]:
d=20
X=np.random.randn(n,d)
eps=np.random.normal(0,1,n)
beta=np.concatenate((np.zeros(d-3),[3,2,4]))

Y=np.dot(X,beta)+eps

## Méthode Ridge

**Implémenter la méthode Ridge à l'aide des formules du cours.**
**Appliquer pour $X$ et $\beta$ définis par:**

    X=np.random.rand(n,d-1)
    X=np.concatenate((np.transpose([2*X[:,0]]),X),axis=1)
    
    beta=np.concatenate(([1],np.zeros(d-1)))

In [ ]:
d=4
n=5
X=np.random.rand(n,d-1)

X=np.concatenate((np.transpose([2*X[:,0]]),X),axis=1)
eps=np.random.normal(0,1,n)
beta=np.concatenate(([1],np.zeros(d-1)))

Y=np.dot(X,beta)+eps
l=1.

**On ne retrouve pas le $\beta$ d'origine. Pourquoi?**

*Réponse:*

## Méthode Elastic Net

La méthode Elastic Net combine les deux pénalisations $\| . \|_1$ et $\| . \|_2$.
On minimise alors:

$$ \beta \mapsto \|Y-X\beta\|_2^2 + \alpha\left(\lambda \|\beta\|_1 + (1-\lambda)\|\beta\|_2^2\right) $$
---------------

dans l'espoir de combiner les deux effets des méthodes LASSO et Ridge.

Cette méthode est codée dans la classe

    linear_model.ElasticNet
    
**Implémenter cette méthode pour un jeu de données simulé où:**
+ $X$ est généré par des variables aléatoires de loi $\mathcal{N}(0,1)$ i.i.d. pour ses $d-3$ premières colonnes, puis les 3 dernières sont des combinaisons linéaires de deux des colonnes précédentes.
+ $\varepsilon$ est une suite de v.a.i.i.d. de loi $\mathcal{N}(0,1)$.
+ $\beta=(1,-3,0,...,0,0,3)$.

On demande de retourner $\hat\beta$ par cette méthode.

**Commenter.**

*Réponse:*